In [102]:
import nltk
nltk.download('inaugural')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import inaugural

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Pre-Processing

## Get Text

In [103]:
# list names inaugural addresses
list_fnames = inaugural.fileids()
# methods inaugural object
print(dir(inaugural))
print(list_fnames)

['CorpusView', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_citation', '_encoding', '_fileids', '_get_root', '_license', '_para_block_reader', '_read_para_block', '_read_sent_block', '_read_word_block', '_readme', '_root', '_sent_tokenizer', '_tagset', '_unload', '_word_tokenizer', 'abspath', 'abspaths', 'citation', 'encoding', 'ensure_loaded', 'fileids', 'license', 'open', 'paras', 'raw', 'readme', 'root', 'sents', 'words']
['1789-Washington.txt', '1793-Washington.txt', '1797-Adams.txt', '1801-Jefferson.txt', '1805-Jefferson.txt', '1809-Madison.txt', '1813-Madison.txt', '1817-Monroe.txt', '1821-Monroe.txt', '1825-Adams.txt', '1829-Jackson.txt', '1833-Jackson.txt', '1837-VanBuren.txt', '184

In [104]:
list_docs = []    # list of raw text documents
for fid in list_fnames:
    list_docs.append(inaugural.raw(fid))

print(len(list_docs))

#print(name_doc)
#print(list_docs[0])

59


In [105]:
# access the words
# Print the first 100 words
print(inaugural.words('1789-Washington.txt')[:20])
 # replace words with paras - get paragraphs

['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':', 'Among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no']


In [106]:
from sklearn.feature_extraction.text import CountVectorizer # extract words and count words in each text
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter

import spacy   # another tokenizer, lemmatizer (has --> be)
nlp = spacy.load('en_core_web_sm')

#1. Build the inverted index as shown in indexing.ipynb (class repository). Use spacy and the ner processing to add names, locations, organizations that span more than one word as terms to your vocabulary (look at the sample code in demo_spacy.ipynb).

##Pre-Process

In [107]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def preProcess(doc):  # text processing for one text
    tokens = nlp(doc)

    # eliminates stop words and non-alpha characters
    terms = [token.lemma_.lower()
             for token in tokens
             if not token.is_stop and token.is_alpha]

    ners = []
    for ner in tokens.ents:
        tokens_ner = nlp(ner.text)
        if len(tokens_ner) > 1 and ner.label_ in ["PERSON", "GPE", "ORG", "FAC"]:
            # remove stop words at the beginning
            if tokens_ner[0].is_stop:
                ner_text = ' '.join([t.text for t in tokens_ner[1:]])
            else:
                ner_text = ner.text
            ners.append(ner_text.lower())

    # Extend terms with ners (this makes it a flat list)
    terms.extend(ners)

    return terms  # returning the flat list directly

all_terms  = []
for doc in list_docs:
  terms_spacy = preProcess(doc)
  all_terms.extend(terms_spacy) # contains all words in the collection in one list
  #all_tokens.append(tokens_spacy) # contains all spacy tokens in each document, as a list of size num documents


In [108]:
# Extract terms (unique) and document frequency (count tokens)
# all_tokens list of tuples (token, doc_id)
def doc_freq(all_tokens):
    dict_doc_freq = {}

    tokens_by_doc = {} # set that stores unique tokens per document

    for (token, doc) in all_tokens: # iterates through all (token, doc_id) tuples
        if doc not in tokens_by_doc:
            tokens_by_doc[doc] = set()  # initializes set for each document
        tokens_by_doc[doc].add(token)  # adds unique token to the set for that document

    for doc, tokens in tokens_by_doc.items(): # calculates document frequency for each unique token
        for token in tokens:
            if token in dict_doc_freq:
                dict_doc_freq[token] += 1
            else:
                dict_doc_freq[token] = 1

    tuples_doc_freq = sorted(dict_doc_freq.items(), key=lambda x: x[0]) # sorts by key (term)

    return dict(tuples_doc_freq)

In [109]:
# Extract term frequency of each term in each document it appears in
# dict_term_freq = {term: {doc1:tf1, doc2:tf2, ...}} # includes only docs that have
# non-zero term frequency
def term_freq(all_tokens, dict_doc_freq):
  dict_term_freq = {term:{} for term in dict_doc_freq.keys()} # initialize dictionary with all unique terms
  for (token, doc) in set(all_tokens): # converted to set to avoid duplicates of terms in documents
    if doc in dict_term_freq[token]:
      dict_term_freq[token][doc] += 1
    else: # if doc is not a key in the dictionary
      dict_term_freq[token][doc] = 1

  return dict_term_freq

In [110]:
# extracts a list of (token, doc_id) from all documents
# input: a list of documents
# output: a list of sorted (token, doc_id) tuples
def extract_token_doc_id(list_doc):
  all_tokens = []
  for ind_doc, doc in enumerate(list_doc):
    tokens_doc = [(token, ind_doc) for token in preProcess(doc)]
    all_tokens.extend(tokens_doc)

  # sorts by token name
  all_tokens = sorted(all_tokens, key = lambda x:x[0])

  return all_tokens

In [111]:
all_tokens = extract_token_doc_id(list_docs) # extract a list of tuples (token, doc id), sorted alphabetically
dict_doc_freq = doc_freq(all_tokens)
dict_term_freq = term_freq(all_tokens, dict_doc_freq) # extracts document frequency

In [112]:
#ner_freq_collec = Counter(all_ners)
#sorted(ner_freq_collec)
term_freq_collec = Counter(all_terms)
sorted(term_freq_collec)

['abandon',
 'abandonment',
 'abate',
 'abdicate',
 'abeyance',
 'abhorring',
 'abide',
 'ability',
 'abject',
 'able',
 'ably',
 'abnormal',
 'abode',
 'abolish',
 'aboriginal',
 'aborigine',
 'abound',
 'abraham',
 'abraham lincoln',
 'abreast',
 'abridge',
 'abroad',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absolutism',
 'absorb',
 'abstain',
 'abstract',
 'abstraction',
 'absurd',
 'abundance',
 'abundant',
 'abundantly',
 'abuse',
 'academy',
 'accept',
 'acceptance',
 'access',
 'accessible',
 'accession',
 'accident',
 'accidental',
 'acclaim',
 'accommodation',
 'accompany',
 'accomplish',
 'accomplishment',
 'accord',
 'accordance',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accounting',
 'accrue',
 'accumulate',
 'accumulation',
 'accurately',
 'accustom',
 'achieve',
 'achievement',
 'acknowledge',
 'acknowledgment',
 'acquaint',
 'acquaintance',
 'acquiesce',
 'acquiescence',
 'acquire',
 'acquisition',
 'acquit',
 'acrimony',
 'act',
 'act

In [113]:
# compute the frequency of all terms in the collection
term_freq_collec = Counter(all_terms)
# how many terms there are in the collection
print("Size of the collection vocabulary", len(term_freq_collec))
# length of the collection = sum length of all documents
print("Length collection", len(all_terms))

Size of the collection vocabulary 6672
Length collection 58643


In [114]:
doc_tokenized = preProcess(inaugural.raw(list_fnames[-2:])) #inaugural.words(doc)
print(doc_tokenized)

['chief', 'justice', 'roberts', 'president', 'carter', 'president', 'clinton', 'president', 'bush', 'president', 'obama', 'fellow', 'americans', 'people', 'world', 'thank', 'citizen', 'america', 'join', 'great', 'national', 'effort', 'rebuild', 'country', 'restore', 'promise', 'people', 'determine', 'course', 'america', 'world', 'year', 'come', 'face', 'challenge', 'confront', 'hardship', 'job', 'year', 'gather', 'step', 'carry', 'orderly', 'peaceful', 'transfer', 'power', 'grateful', 'president', 'obama', 'lady', 'michelle', 'obama', 'gracious', 'aid', 'transition', 'magnificent', 'thank', 'today', 'ceremony', 'special', 'meaning', 'today', 'merely', 'transfer', 'power', 'administration', 'party', 'transfer', 'power', 'washington', 'dc', 'give', 'people', 'long', 'small', 'group', 'nation', 'capital', 'reap', 'reward', 'government', 'people', 'bear', 'cost', 'washington', 'flourish', 'people', 'share', 'wealth', 'politician', 'prosper', 'job', 'leave', 'factory', 'close', 'establishme

## Create The Index

In [115]:
#list_docs = []    # list of raw text documents
#for fid in list_fnames:
#    list_docs.append(inaugural.raw(fid))

#print(len(list_docs))
#print(list_fnames)
#print(inaugural.words('1789-Washington.txt')) # replace words with paras - get paragraphs


def create_invIndex(docList):
    invIndex = {}
    for doc in docList:
        doc_tokenized = preProcess(inaugural.raw(doc)) #inaugural.words(doc)
        for term in doc_tokenized:
            # Check if the term is in the dictionary, and if not, initialize an empty list
            if term not in invIndex:
                invIndex[term] = []  # Initialize an empty list for the term

            # Append the document to the list if it is not already present
            if doc not in invIndex[term]:
                invIndex[term].append(doc)

    return invIndex


In [116]:
myInvIndex = create_invIndex(list_fnames)

#for term, docs in myInvIndex.items():
#       print(f"{term}: {docs}")

## Which nlp processing steps did you use?


*   remove stop words
*   remove non-alpha characters
*   add ners (Person, group, organization, locations)
*   make all terms lowercase





##What is the vocabulary size (# of unique terms after processing) of the whole corpus after nlp processing (number of unique terms).

In [117]:
len(myInvIndex)

6672


##Which names, locations and organizations you found and added to the corpus vocabulary
(examples of what was added to corpus vocabulary)

*   abraham lincoln
*   arlington cemetery
*   saint augustine
*   kamala harris





##What are the most frequent and least frequent 10 words in the corpus and their document frequencies.

In [118]:
def termFrequency(docList):
  freqList = {}
  for doc in docList:
    doc_tokenized = preProcess(inaugural.raw(doc)) #inaugural.words(doc)
    for term in doc_tokenized:
        # Check if the term is in the dictionary, and if not, initialize an empty list
        if term not in freqList:
            freqList[term] = 1  # Initialize an empty list for the term
        # Append the document to the list if it is not already present
        else:
            freqList[term] += 1
  return freqList

In [119]:
def sort_by_frequency(freq_dict):
    sorted_freq = sorted(freq_dict.items(), key=lambda item: item[1], reverse=True) # sorts the dictionary by values in descending order
    return sorted_freq

In [120]:
myFreqList = termFrequency(list_fnames)

my_sorted = sort_by_frequency(myFreqList)

print("Most Common")
print("Term , Frequency, Document Frequency")
for term, freq in my_sorted[:10]:
       print(f"{term} , {freq} , {len(myInvIndex[term])}")

print("\n")


print("Least Common")
print("Term , Frequency, Document Frequency")
for term, freq in my_sorted[-10:]:
       print(f"{term} , {freq} , {len(myInvIndex[term])}")

Most Common
Term , Frequency, Document Frequency
government , 651 , 55
people , 633 , 57
nation , 517 , 57
great , 441 , 57
country , 359 , 56
world , 348 , 54
power , 341 , 50
shall , 316 , 51
states , 309 , 46
citizen , 298 , 55


Least Common
Term , Frequency, Document Frequency
systemic , 1 , 1
cascade , 1 , 1
anthem , 1 , 1
verse , 1 , 1
forebearer , 1 , 1
american wayârestless , 1 , 1
emancipation proclamation , 1 , 1
kamala harris , 1 , 1
arlington cemetery , 1 , 1
saint augustine , 1 , 1


#2. Come up with 3 queries related to the inaugural addresses (look at the text of some of the addresses). The queries should consists of more than one word. For example: 'American foreign policy', 'recover from economic depression', etc. Check that all words in your queries appear somewhere in your documents. Hint: Do not use very common or very rare words. Do not forget to process the queries using the same nlp processing steps as the documents.



##For each query, display the query and the document frequency of each term.

In [121]:
def queryProcess(query, invIndex):
    tokens = preProcess(query.lower())  # Ensure lowercase before tokenizing
    print("Tokens:", tokens)  # Debug step
    for token in tokens:
        if token in invIndex:
            addresses = []
            print(f"{token} , {len(invIndex[token])}")
            for address in invIndex[token]:
                addresses.append(address)
            print(addresses)
        else:
            print(f"Token '{token}' not found in index")


In [122]:
Query_1 = "economic growth policy"
Query_2 = "national security defense"
Query_3 = "god bless america"

print(f"Query 1: {Query_1}")
print(queryProcess(Query_1,myInvIndex))

print("\n")


print(f"Query 2: {Query_2}")
print(queryProcess(Query_2,myInvIndex))

print("\n")

print(f"Query 3: {Query_3}")
print(queryProcess(Query_3,myInvIndex))

Query 1: economic growth policy
Tokens: ['economic', 'growth', 'policy']
economic , 16
['1897-McKinley.txt', '1909-Taft.txt', '1913-Wilson.txt', '1917-Wilson.txt', '1921-Harding.txt', '1925-Coolidge.txt', '1929-Hoover.txt', '1933-Roosevelt.txt', '1937-Roosevelt.txt', '1949-Truman.txt', '1953-Eisenhower.txt', '1957-Eisenhower.txt', '1981-Reagan.txt', '1985-Reagan.txt', '2005-Bush.txt', '2013-Obama.txt']
growth , 19
['1809-Madison.txt', '1817-Monroe.txt', '1841-Harrison.txt', '1853-Pierce.txt', '1857-Buchanan.txt', '1881-Garfield.txt', '1889-Harrison.txt', '1893-Cleveland.txt', '1905-Roosevelt.txt', '1909-Taft.txt', '1929-Hoover.txt', '1949-Truman.txt', '1957-Eisenhower.txt', '1965-Johnson.txt', '1969-Nixon.txt', '1981-Reagan.txt', '1985-Reagan.txt', '2005-Bush.txt', '2009-Obama.txt']
policy , 34
['1789-Washington.txt', '1809-Madison.txt', '1813-Madison.txt', '1817-Monroe.txt', '1821-Monroe.txt', '1825-Adams.txt', '1829-Jackson.txt', '1833-Jackson.txt', '1837-VanBuren.txt', '1841-Harriso

# 3. Write a function to compute the similarity of a query with the collection of inaugural addresses. Do this efficiently by computing the similarity between a query and only the set of documents that contain at least two words from the query. Implement two different TF-IDF similarity functions (one of them should be BM25 - choose another one from the slides or textbook). Retrieve the ranked list of the first 5 most similar inaugural addresses for each of your queries and for each similarity function.



##For each method, and each query, display the most similar 5 titles of the inaugural addresses (not the full text) and their similarity values.

In [123]:
def documentDict(query, invIndex):
    tokens = preProcess(query.lower())
    addressFreq = {}

    for token in tokens:
        if token in invIndex:  # Check if token exists in the inverted index
            for address in invIndex[token]:
                if address not in addressFreq:
                    addressFreq[address] = 1
                else:
                    addressFreq[address] += 1

    # Only keep addresses that appear for at least two tokens
    addressFreq = {address: freq for address, freq in addressFreq.items() if freq >= 2}

    return addressFreq


In [124]:
print(f"Query 1: {Query_1}")
docList1 = documentDict(Query_1,myInvIndex)
print(documentDict(Query_1,myInvIndex))

print("\n")

print(f"Query 2: {Query_2}")
docList2 = documentDict(Query_2,myInvIndex)
print(documentDict(Query_2,myInvIndex))

print("\n")

print(f"Query 3: {Query_3}")
docList3 = documentDict(Query_3,myInvIndex)
print(documentDict(Query_3,myInvIndex))


Query 1: economic growth policy
{'1897-McKinley.txt': 2, '1909-Taft.txt': 3, '1913-Wilson.txt': 2, '1921-Harding.txt': 2, '1925-Coolidge.txt': 2, '1929-Hoover.txt': 3, '1933-Roosevelt.txt': 2, '1949-Truman.txt': 3, '1953-Eisenhower.txt': 2, '1957-Eisenhower.txt': 2, '1981-Reagan.txt': 2, '1985-Reagan.txt': 2, '2005-Bush.txt': 3, '1809-Madison.txt': 2, '1817-Monroe.txt': 2, '1841-Harrison.txt': 2, '1853-Pierce.txt': 2, '1857-Buchanan.txt': 2, '1881-Garfield.txt': 2, '1889-Harrison.txt': 2, '1893-Cleveland.txt': 2}


Query 2: national security defense
{'1789-Washington.txt': 2, '1797-Adams.txt': 2, '1813-Madison.txt': 2, '1817-Monroe.txt': 3, '1821-Monroe.txt': 2, '1825-Adams.txt': 3, '1829-Jackson.txt': 2, '1837-VanBuren.txt': 2, '1841-Harrison.txt': 3, '1845-Polk.txt': 2, '1853-Pierce.txt': 3, '1857-Buchanan.txt': 3, '1861-Lincoln.txt': 2, '1869-Grant.txt': 2, '1885-Cleveland.txt': 2, '1889-Harrison.txt': 3, '1893-Cleveland.txt': 2, '1901-McKinley.txt': 2, '1909-Taft.txt': 3, '1921-Har

##Bm25

In [125]:
#pip install rank_bm25
!pip install rank_bm25
# Download necessary resources
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [126]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

def BM25(query,documentList):
  tokenized_documents = [preProcess(inaugural.raw(doc)) for doc in documentList]
  bm25 = BM25Okapi(tokenized_documents)
  tokenized_query = preProcess(query)
  scores = bm25.get_scores(tokenized_query)

  doc_scores = list(zip(documentList, scores))
  sorted_doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
  return sorted_doc_scores


In [127]:
print(BM25(Query_1,docList1))
print("\n)")
print(BM25(Query_2,docList2))
print("\n)")
print(BM25(Query_3,docList3))
print("\n)")

[('1929-Hoover.txt', 2.4375133483866036), ('1909-Taft.txt', 2.031863469239845), ('1949-Truman.txt', 2.022562764637589), ('2005-Bush.txt', 1.8428023163837928), ('1921-Harding.txt', 1.7181354751407691), ('1985-Reagan.txt', 1.7081279350211274), ('1925-Coolidge.txt', 1.6873666018940816), ('1981-Reagan.txt', 1.642657941170747), ('1889-Harrison.txt', 1.5434277385663342), ('1933-Roosevelt.txt', 1.4427740790945358), ('1897-McKinley.txt', 1.4080568962451037), ('1953-Eisenhower.txt', 1.4067641957458028), ('1809-Madison.txt', 1.331909201112408), ('1881-Garfield.txt', 1.270467408383716), ('1817-Monroe.txt', 1.2442394730989779), ('1913-Wilson.txt', 1.2047353035052386), ('1957-Eisenhower.txt', 1.203713672573131), ('1857-Buchanan.txt', 1.1893648820969989), ('1853-Pierce.txt', 1.126601031941811), ('1893-Cleveland.txt', 1.10486641418646), ('1841-Harrison.txt', 0.7451835468346131)]

)
[('1949-Truman.txt', 3.261278435466696), ('1985-Reagan.txt', 3.0896473869156065), ('1825-Adams.txt', 3.027766974371089),

In [128]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

##Cosine Similarity

In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosSim(query, documentList):

    # Preprocess the query and documents
    tokenized_query = preProcess(query)  # Ensure preProcess returns meaningful tokens
    tokenized_docList = [preProcess(inaugural.raw(doc)) for doc in documentList]  # Preprocess document content

    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Combine the query and documents into a single list of space-separated strings
    all_docs = [' '.join(tokenized_query)] + [' '.join(doc) for doc in tokenized_docList]

    # Fit the vectorizer and transform into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_docs)

    # Get the query vector (first row) and document vectors (remaining rows)
    query_vector = tfidf_matrix[0]  # The first vector corresponds to the query
    doc_vectors = tfidf_matrix[1:]  # The rest correspond to the documents

    # Compute cosine similarity between the query and each document
    similarities = cosine_similarity(query_vector, doc_vectors).flatten()

    # Pair each document with its respective similarity score
    doc_scores = list(zip(documentList, similarities))

    # Sort the documents by their similarity scores in descending order
    sorted_doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    return sorted_doc_scores


In [130]:
print(cosSim(Query_1,docList1))
print("\n")
print(cosSim(Query_2,docList2))
print("\n)")
print(cosSim(Query_3,docList3))
print("\n)")

[('1929-Hoover.txt', 0.06557755153606054), ('1949-Truman.txt', 0.05988859735919958), ('1909-Taft.txt', 0.05555297475594834), ('1985-Reagan.txt', 0.05083523572979329), ('1921-Harding.txt', 0.0508045142590452), ('1925-Coolidge.txt', 0.050502728238718445), ('1981-Reagan.txt', 0.04979102301979427), ('1897-McKinley.txt', 0.04252545400911965), ('1953-Eisenhower.txt', 0.04080788142341073), ('1889-Harrison.txt', 0.03989617863812912), ('1933-Roosevelt.txt', 0.031989902113003205), ('2005-Bush.txt', 0.028092525639517956), ('1809-Madison.txt', 0.02487790435779317), ('1881-Garfield.txt', 0.02339821574149669), ('1957-Eisenhower.txt', 0.021048730874658653), ('1913-Wilson.txt', 0.018932202091797086), ('1817-Monroe.txt', 0.018911910245236193), ('1853-Pierce.txt', 0.01690795124577483), ('1857-Buchanan.txt', 0.016757017291487923), ('1893-Cleveland.txt', 0.01556535960817294), ('1841-Harrison.txt', 0.007112597008058642)]


[('1949-Truman.txt', 0.0812118936532361), ('1985-Reagan.txt', 0.07375126805998652), 

#4. For each similarity method, and each query, compute the precision in the following way: Read the text of the first 3 addresses in the ranked list for each query. Using your own judgement, rate each text as either relevant or irrelevant to the query. Count how many you rated relevant and divide it by three. This is the precision:

# Precision(query_x, method_y) = relevant texts / first 3 ranked docs

* Reagan 85
 * national security defense
   * Relevance: Relevant
  * god bless america
   * Relevance: Relevant
* Hoover
  * economic growth policy
   * Relevance: Not Relevant
* Taft
  * economic growth policy
   * Relevance: Relevant
* Truman
  * economic growth policy
   * Relevance: Relevant
  * national security defense
   * Relevance: Relevant
* Adams
  * national security defense
   * Relevance: Relevant
* Trump
  * god bless america
   * Relevance: Relevant
* Bush
  * god bless america
   * Relevance: Relevant
* Clinton
  * god bless america
   * Relevance: Relevant
* Nixon
  * god bless america:
   * Relevance: Not Relevant



## Display the precision values for each query and each method.

In [131]:
print(f"Precision for Query 1, {Query_1}, on BM25: {2/3}")
print(f"Precision for Query 1, {Query_1}, on Cosine Similarity: {2/3}")

print("\n")

print(f"Precision for Query 2, {Query_2}, on BM25: {3/3} ")
print(f"Precision for Query 2, {Query_2}, on Cosine Similarity: {3/3}")

print("\n")

print(f"Precision for Query 3, {Query_3}, on BM25: {3/3} ")
print(f"Precision for Query 3, {Query_3}, on Cosine Similarity: {2/3}")

Precision for Query 1, economic growth policy, on BM25: 0.6666666666666666
Precision for Query 1, economic growth policy, on Cosine Similarity: 0.6666666666666666


Precision for Query 2, national security defense, on BM25: 1.0 
Precision for Query 2, national security defense, on Cosine Similarity: 1.0


Precision for Query 3, god bless america, on BM25: 1.0 
Precision for Query 3, god bless america, on Cosine Similarity: 0.6666666666666666


# 5. Add length normalization to your methods. First, compute the length of the collection of speeches, then find the max, min, average, and standard deviation. Then choose the parameters of your length normalization method according to these values.

## Compute the similarity between each query and the collection using length normalization in your methods.


In [132]:
#from os import ST_RDONLY
import statistics

#find the length of each speech, and put it into a list
def lengthCollection(docList):
  lengthList = []
  for doc in docList:
    doc_tokenized = preProcess(inaugural.raw(doc)) #inaugural.words(doc)
    lengthList.append(len(doc_tokenized))
  return lengthList

#find the max, min, and stdv
lengthList = lengthCollection(list_fnames)
max_length = max(lengthList)
min_length = min(lengthList)
avg_length = sum(lengthList) / len(lengthList)
std_dev = statistics.stdev(lengthList)

print(f"max_length: {max_length}")
print(f"min_length: {min_length}")
print(f"avg_length: {avg_length}")
print(f"std_dev: {std_dev}")

max_length: 3322
min_length: 56
avg_length: 993.9491525423729
std_dev: 569.7383980368054


## Cosine Normalized

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def normalize_by_mean_length(vectors, lengths, mean_length):
    """
    Normalize each document vector by dividing by the mean length.
    """
    normalized_vectors = vectors / mean_length  # Element-wise division by mean length
    return normalized_vectors

def normalize_by_std_dev(vectors, lengths, mean_length, std_dev):
    """
    Normalize document length by Z-score (subtract mean and divide by standard deviation).
    """
    normalized_lengths = (lengths - mean_length) / std_dev
    return vectors / normalized_lengths[:, np.newaxis]

def cosSimNormal(query, documentList, normalization='mean'):
    # Preprocess the query and documents
    tokenized_query = preProcess(query)  # Ensure preProcess returns meaningful tokens
    tokenized_docList = [preProcess(inaugural.raw(doc)) for doc in documentList]  # Preprocess document content

    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Combine the query and documents into a single list of space-separated strings
    all_docs = [' '.join(tokenized_query)] + [' '.join(doc) for doc in tokenized_docList]

    # Fit the vectorizer and transform into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_docs).toarray()  # Convert sparse matrix to dense array

    # Compute the length of each document (number of terms)
    doc_lengths = np.array([len(doc.split()) for doc in all_docs])

    # Calculate max, mean, and std dev for document lengths
    max_length = np.max(doc_lengths)
    mean_length = np.mean(doc_lengths)
    std_dev = np.std(doc_lengths)

    # Apply chosen normalization
    if normalization == 'mean':
        tfidf_matrix_normalized = normalize_by_mean_length(tfidf_matrix, doc_lengths, mean_length)
    elif normalization == 'std_dev':
        tfidf_matrix_normalized = normalize_by_std_dev(tfidf_matrix, doc_lengths, mean_length, std_dev)
    # Add other normalization methods as needed, like max or min-max scaling

    # Get the query vector (first row) and document vectors (remaining rows)
    query_vector = tfidf_matrix_normalized[0]  # The first vector corresponds to the query
    doc_vectors = tfidf_matrix_normalized[1:]  # The rest correspond to the documents

    # Compute cosine similarity between the query and each document
    similarities = cosine_similarity(query_vector.reshape(1, -1), doc_vectors).flatten()

    # Pair each document with its respective similarity score
    doc_scores = list(zip(documentList, similarities))

    # Sort the documents by their similarity scores in descending order
    sorted_doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    return sorted_doc_scores


In [134]:
print(cosSimNormal(Query_1,docList1))
print("\n")
print(cosSimNormal(Query_2,docList2))
print("\n)")
print(cosSimNormal(Query_3,docList3))
print("\n)")

[('1929-Hoover.txt', 0.06557755153606029), ('1949-Truman.txt', 0.05988859735919968), ('1909-Taft.txt', 0.05555297475594809), ('1985-Reagan.txt', 0.05083523572979303), ('1921-Harding.txt', 0.05080451425904531), ('1925-Coolidge.txt', 0.050502728238718264), ('1981-Reagan.txt', 0.04979102301979425), ('1897-McKinley.txt', 0.042525454009119744), ('1953-Eisenhower.txt', 0.0408078814234108), ('1889-Harrison.txt', 0.03989617863812901), ('1933-Roosevelt.txt', 0.031989902113003184), ('2005-Bush.txt', 0.028092525639518015), ('1809-Madison.txt', 0.02487790435779314), ('1881-Garfield.txt', 0.02339821574149676), ('1957-Eisenhower.txt', 0.02104873087465868), ('1913-Wilson.txt', 0.0189322020917971), ('1817-Monroe.txt', 0.018911910245236193), ('1853-Pierce.txt', 0.016907951245774877), ('1857-Buchanan.txt', 0.01675701729148787), ('1893-Cleveland.txt', 0.01556535960817294), ('1841-Harrison.txt', 0.007112597008058601)]


[('1949-Truman.txt', 0.08121189365323606), ('1985-Reagan.txt', 0.07375126805998654), (

##BM25 With Normalization

In [135]:
import numpy as np

# term frequency function for TF-IDF
def tf_func(tf_freq_doc):
    return np.log(tf_freq_doc + 1)

# inverse document frequency function
def idf_func(tf_doc_freq, nr_docs):
    return np.log((nr_docs + 1) / tf_doc_freq)


In [136]:
# BM25 parameters
k1 = 1.5 # controls influence of term frequency on rank score
b = 0.75 # controls degree of document length normalization
avg_doc_len = sum([len(preProcess(doc)) for doc in list_docs]) / len(list_docs)

def bm25_tf_func(tf_freq_doc, doc_len, avg_doc_len, k1, b): # BM25 term frequency function
    return (tf_freq_doc * (k1 + 1)) / (tf_freq_doc + k1 * (1 - b + b * (doc_len / avg_doc_len)))

In [137]:
# Computes TF-IDF similarity
def tf_idf(query, dict_doc_freq, dict_term_freq, nr_docs):
    tokens = preProcess(query) # processes query using the same NLP steps
    tf_query = Counter(tokens)

    sim_query_doc = {}  # doc_id: similarity value

    for term_q, tf_term_q in tf_query.items(): # calculates similarity for each document containing query terms
        if term_q in dict_term_freq:
            for doc_id, tf_doc in dict_term_freq[term_q].items():
                tf_doc_value = tf_func(tf_doc)
                idf_value = idf_func(dict_doc_freq[term_q], nr_docs)
                if doc_id in sim_query_doc:
                    sim_query_doc[doc_id] += tf_term_q * tf_doc_value * idf_value
                else:
                    sim_query_doc[doc_id] = tf_term_q * tf_doc_value * idf_value

    sorted_sim = dict(sorted(sim_query_doc.items(), key=lambda x: x[1], reverse=True)) # sorts the documents by similarity

    return sorted_sim

In [138]:
def bm25_ln(query, dict_doc_freq, dict_term_freq, nr_docs):
    tokens = preProcess(query)  # processes query using the same NLP steps
    tf_query = Counter(tokens)

    sim_query_doc = {}  # doc_id: similarity value

    for term_q, tf_term_q in tf_query.items():
        if term_q in dict_term_freq:
            for doc_id, tf_doc in dict_term_freq[term_q].items():
                doc_len = len(preProcess(list_docs[doc_id]))
                tf_doc_value = bm25_tf_func(tf_doc, doc_len, avg_doc_len, k1, b)
                idf_value = idf_func(dict_doc_freq[term_q], nr_docs)

                if doc_id in sim_query_doc:
                    sim_query_doc[doc_id] += tf_term_q * tf_doc_value * idf_value
                else:
                    sim_query_doc[doc_id] = tf_term_q * tf_doc_value * idf_value

    sorted_sim = dict(sorted(sim_query_doc.items(), key=lambda x: x[1], reverse=True))

    return sorted_sim

In [139]:
# retrieves the top 5 most similar documents for each query and similarity function
def get_top_5_resultsDavid(query, sim_func, method_name):
    print(f"\nMethod: {method_name}, Query: '{query}'")
    sorted_sim = sim_func(query, dict_doc_freq, dict_term_freq, len(list_docs))
    top_5_docs = list(sorted_sim.items())[:5]  # Gets top 5
    for doc_id, sim_value in top_5_docs:
        print(f"Document: {list_fnames[doc_id]}, Similarity: {sim_value}")

In [140]:
def get_top_5_resultsDaniel(query,sim_func,doc_list):
  print(f"{query}:")
  return sim_func(query,doc_list)[:5]

In [141]:
queries = [
    "economic growth policy",
    "national security defense",
    "god bless america"
]

docLists =[
    docList1,
    docList2,
    docList3
]

In [ ]:
# computes length-normalized similarity for each query
for query in queries:
    get_top_5_resultsDavid(query, bm25_ln, "BM25 with Length Normalization")
print("\nBM25 without Normalization:")
for query, docList in zip(queries, docLists):
  print(get_top_5_resultsDaniel(query,BM25,docList))
print("\nCosine with Length Normalization:")
for query, docList in zip(queries, docLists):
  print(get_top_5_resultsDaniel(query,cosSimNormal,docList))
print("\nCosine Without Normalization:")
for query, docList in zip(queries, docLists):
  print(get_top_5_resultsDaniel(query,cosSim,docList))


Method: BM25 with Length Normalization, Query: 'economic growth policy'
Document: 2005-Bush.txt, Similarity: 3.112858112654888
Document: 1949-Truman.txt, Similarity: 2.947500570447289
Document: 1957-Eisenhower.txt, Similarity: 2.8407067356116418
Document: 1981-Reagan.txt, Similarity: 2.4582494741395444
Document: 1929-Hoover.txt, Similarity: 2.4108844978274293

Method: BM25 with Length Normalization, Query: 'national security defense'
Document: 1941-Roosevelt.txt, Similarity: 2.3153965463514936
Document: 2001-Bush.txt, Similarity: 2.1556574080385538
Document: 2013-Obama.txt, Similarity: 1.9400739650749954
Document: 1957-Eisenhower.txt, Similarity: 1.8244835024775117
Document: 1949-Truman.txt, Similarity: 1.7969847563466097

Method: BM25 with Length Normalization, Query: 'god bless america'
Document: 1945-Roosevelt.txt, Similarity: 3.2379022720670356
Document: 2017-Trump.txt, Similarity: 2.5321842925087528
Document: 1993-Clinton.txt, Similarity: 2.4586840969493116
Document: 2001-Bush.tx

## Discuss any changes you see to the top 5 for each query and method and the precision results. Did the results improve, stay the same, or worsen. Be specific.

### After adding length normalization to both our BM25 and cosine similarity methods, we got a new set of documents as the top 5 for each query. Specifically, for our BM25 method when we ran it without length normalization the documents we got back as the top 5 for the query "economic growth policy" were Hoover, Taft, Truman, Bush, and Harding, and with length normalization added, the top 5 documents were instead Bush, Truman, Eisenhower, Reagan, and Hoover. The similarity results were much higher for the query using BM25 with length normalization than without, going from 2.3 all the way to 3.1 for the 1st document in the top 5 for the query. When we ran BM25 without length normalization for the query "national security defense" the top 5 documents we got back were Truman, Reagan, Adams, Harrison, and Roosevelt, and once we added length normalization the top 5 changed to Roosevelt, Bush, Obama, Eisenhower, and Truman. The similarity results were lower for the query using BM25 with length normalization than without, going from 3.26 all the way to 2.3 for the 1st document in the top 5 for the query. For our third query "god bless america", we found that using BM25 without length normalization gave back the top 5 documents as Trump, Reagan, Bush, Obama(2013), Obama(2009). Once we added length normalization, the top 5 for the query "god bless america" were Roosevelt, Trump, Clinton, Bush, and Obama(2013). The similarity results were much higher for the query using BM25 with length normalization than without, going from 3.59 all the way to 3.23 for the 1st document in the top 5 for the query. For our cosine similarity method the top 5 relevant documents did not change after length normalization, as the similarity results stayed the same so the top 5 for the query "economic growth policy" were Hoover, Truman, Taft, Reagan, and Harding. Using cosine similarity for the query "national security defense" gave us the top 5 documents as Truman, Reagan, Adams, Monroe, and Pierce. Additionally, using cosine similarity for the third query "god bless america" gave us the top 5 documents as Trump, Clinton, Nixon, Biden, and Bush. When it comes to the precision results that we got, the results we got for the BM25 results did worsen a bit as for the first query the precision result was 0.66 and after normalization it went down to 0.33, but other than that the precision results were consistent for the other 2 queries. With the cosine similarity method, the precision results did improve as the first query had a precision result of 0.66 and it went up to 1.0 after normalization, in which the precision results stayed consistent for the other 2 queries.

#Cosine Mean
##The results stayed Mostly the same

#Cosine Std_dev

##I got an entirely new set of documents

* Reagan 81
 * national security defense
   * Relevance: Relevant
* Reagan 85
 * national security defense
   * Relevance: Relevant
 * economic growth policy
   * Relevence: Relevant
* Truman
  * economic growth policy
   * Relevance: Relevant
  * national security defense
   * Relevance: Relevant
* Jackson
  * national security defense
   * Relevance: Relevent
* Trump
  * god bless america
   * Relevance: Relevant
* Clinton
  * god bless america
   * Relevance: Relevant
* Nixon
  * god bless america:
   * Relevance: Not Relevant

#BM25 Mean

##I got an entirely new set of documents

# 6. Compare and discuss the precision results you obtained with the two TF-IDF similarity functions you used with and without length normalization. Which method gave you better results and why.

In [ ]:
print("Without Normalization \n")

print(f"Precision for Query 1, {Query_1}, on Cosine Similarity: {2/3} \n")
print(f"Precision for Query 1, {Query_1}, on BM25 Similarity: {2/3} \n")

print(f"Precision for Query 2, {Query_2}, on Cosine Similarity: {3/3} \n")
print(f"Precision for Query 2, {Query_2}, on BM25 Similarity: {3/3} \n")

print(f"Precision for Query 3, {Query_3}, on Cosine Similarity: {2/3} \n")
print(f"Precision for Query 3, {Query_3}, on BM25 Similarity: {3/3} \n")

print("With Normalization \n")

print(f"Precision for Query 1, {Query_1}, on Cosine Similarity: {3/3} \n")
print(f"Precision for Query 1, {Query_1}, on BM25 Similarity: {1/3} \n")

print(f"Precision for Query 2, {Query_2}, on Cosine Similarity: {3/3} \n")
print(f"Precision for Query 2, {Query_2}, on BM25 Similarity: {3/3} \n")

print(f"Precision for Query 3, {Query_3}, on Cosine Similarity: {2/3} \n")
print(f"Precision for Query 3, {Query_3}, on BM25 Similarity: {3/3} \n")

## New Section

# Part 5 -
### New Section